In [1]:
import web3

In [2]:
from web3 import Web3, HTTPProvider

In [3]:
# forge script script/Counter.s.sol:CounterScript --rpc-url http://localhost:8545 --private-key 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80 --broadcast

# cast call 0x5FbDB2315678afecb367f032d93F642f64180aa3 "number()(uint)" --rpc-url http://localhost:8545

# cast send 0x5FbDB2315678afecb367f032d93F642f64180aa3 "increment()()" --rpc-url http://localhost:8545 --private-key 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80

# deploy CP.sol:

# forge create --rpc-url http://localhost:8545 --private-key 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80 src/ProgrammingCompetition.sol:ProgrammingCompetition --broadcast

# deploy AdditionSolution.sol:

# forge create --rpc-url http://localhost:8545 --private-key 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80 src/AdditionSolution.sol:AdditionSolution --broadcast

In [4]:
"""
function registerProblem(
        uint256 problemId,
        bytes32 expectedResultsHash,
        bytes[] calldata testCases
    ) external payable onlyOwner {
        require(
            problems[problemId].expectedResultsHash == bytes32(0),
            "Problem already exists"
        );
        require(msg.value > 0, "Prize must be greater than zero");
        require(testCases.length > 0, "Must provide at least one test case");

        problems[problemId] = Problem({
            expectedResultsHash: expectedResultsHash,
            prize: msg.value,
            isSolved: false,
            testCases: testCases
        });

        emit ProblemRegistered(problemId, msg.value);
    }
"""

# 0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9

'\nfunction registerProblem(\n        uint256 problemId,\n        bytes32 expectedResultsHash,\n        bytes[] calldata testCases\n    ) external payable onlyOwner {\n        require(\n            problems[problemId].expectedResultsHash == bytes32(0),\n            "Problem already exists"\n        );\n        require(msg.value > 0, "Prize must be greater than zero");\n        require(testCases.length > 0, "Must provide at least one test case");\n\n        problems[problemId] = Problem({\n            expectedResultsHash: expectedResultsHash,\n            prize: msg.value,\n            isSolved: false,\n            testCases: testCases\n        });\n\n        emit ProblemRegistered(problemId, msg.value);\n    }\n'

In [5]:
RPC = "http://localhost:8545"

CONTRACT_ADDRESS = "0x5FC8d32690cc91D4c39d9d3abcBD16989F875707"

w3 = Web3(HTTPProvider(RPC))

In [6]:
from eth_abi import encode
from eth_utils import keccak

# Encode each result as Solidity would (abi.encode(uint256))
encoded_2 = encode(['uint256'], [2])
encoded_4 = encode(['uint256'], [4])
encoded_12 = encode(['uint256'], [12])

# Concatenate as abi.encodePacked would do
all_results = encoded_2 + encoded_4 + encoded_12

# Compute the keccak256 hash
expected_results_hash = keccak(all_results).hex()

print(expected_results_hash)
# Should print: 0x1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856

1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856


In [7]:
from web3 import Web3, HTTPProvider
# from eth_account import Account

# --- Setup ---
RPC = "http://localhost:8545"
CONTRACT_ADDRESS = "0x5FC8d32690cc91D4c39d9d3abcBD16989F875707"
OWNER_PRIVATE_KEY = "0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80"
# OWNER_ADDRESS = Account.from_key(OWNER_PRIVATE_KEY).address
OWNER_ADDRESS ="0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266"

# Replace with your contract's ABI
CONTRACT_ABI = [
    {"type":"function","name":"registerProblem","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"},{"name":"expectedResultsHash","type":"bytes32","internalType":"bytes32"},{"name":"testCases","type":"bytes[]","internalType":"bytes[]"}],"outputs":[],"stateMutability":"payable"},
    {"type":"function","name":"problemExists","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},
    {"type":"function","name":"isProblemSolved","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"bool","internalType":"bool"}],"stateMutability":"view"},
    {"type":"function","name":"getProblemPrize","inputs":[{"name":"problemId","type":"uint256","internalType":"uint256"}],"outputs":[{"name":"","type":"uint256","internalType":"uint256"}],"stateMutability":"view"}
]

w3 = Web3(HTTPProvider(RPC))
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)

# --- Prepare arguments ---
problem_id = 1
expected_results_hash = "0x1f7d96dc808e221cdcfee683a0ce9dfc707f5a1ee10ec6febbcb3621cb1ac856"

# Example: test cases for addition (1+1, 2+2, 5+7)

test_case_1 = encode(['uint256', 'uint256'], [1, 1])
test_case_2 = encode(['uint256', 'uint256'], [2, 2])
test_case_3 = encode(['uint256', 'uint256'], [5, 7])

test_cases = [
    test_case_1,
    test_case_2,
    test_case_3
]

# --- Build transaction ---
tx = contract.functions.registerProblem(
    problem_id,
    expected_results_hash,
    test_cases
).build_transaction({
    'from': OWNER_ADDRESS,
    'value': w3.to_wei(0.1, 'ether'),
    'nonce': w3.eth.get_transaction_count(OWNER_ADDRESS),
    'gas': 500000,
    'gasPrice': w3.to_wei('10', 'gwei')
})

try:
    gas_estimate = w3.eth.estimate_gas({
        'from': tx['from'],
        'to': tx['to'],
        'data': tx['data'],
        'value': tx['value'],
    })
    print(f"Estimated gas: {gas_estimate}")
except Exception as e:
    # Extract revert reason if present
    message = str(e)
    if "revert reason:" in message:
        reason = message.split("revert reason:")[1].strip()
        print(f"Revert reason: {reason}")
    else:
        print(f"Transaction would revert. Error: {message}")

# --- Sign and send ---
signed_tx = w3.eth.account.sign_transaction(tx, private_key=OWNER_PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
print(f"Transaction sent: {tx_hash.hex()}")

# to get the status of the tx we need the receipt:
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
if receipt.status == 1:
    print("Transaction was successful!")
else:
    print("Transaction failed!")

Estimated gas: 301577
Transaction sent: a33e6ebc6492dbdf8bbcabeb4f6ce1b7771d51daf1f75026895a07f70cb09d86
Transaction was successful!


In [8]:
"""
Now we'll call:

function getProblemPrize(uint256 problemId) external view returns (uint256) {
        return problems[problemId].prize;
    }

    function isProblemSolved(uint256 problemId) external view returns (bool) {
        return problems[problemId].isSolved;
    }

    function problemExists(uint256 problemId) external view returns (bool) {
        return problems[problemId].expectedResultsHash != bytes32(0);
    }

"""

# Call the contract to check if the problem exists
problem_id = 1
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")
    
    # Get the prize for the problem
    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    
    # Check if the problem is solved
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 0.1 ETH
Is problem 1 solved? No


In [26]:
!cast balance 0x8A791620dd6260079BF849Dc5567aDC3F2FdC318 --rpc-url http://localhost:8545

10000000000000000000


In [25]:
!cast send 0x8A791620dd6260079BF849Dc5567aDC3F2FdC318 --value 10000000000000000000 --from 0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC --private-key 0x5de4111afa1a4b94908f83103eb1f1706367c2e68ca870fc3fb9a804cdab365a --rpc-url http://0.0.0.0:8545


blockHash            0x4bc508f03836f054ccb7a396217f83b2757a35cd21d366c582d9e8b8d1293796
blockNumber          12
contractAddress      
cumulativeGasUsed    21055
effectiveGasPrice    238415285
from                 0x3C44CdDdB6a900fa2b585dd299e03d12FA4293BC
gasUsed              21055
logs                 []
logsBloom            0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                 
status               1 (success)
transactionHash      0xe4e02b3fe48cbb5436d555b9cfa421c62590d40ed015cbe23e8a2ab32054769a
transactionIn

In [22]:
!cast balance 0x5FC8d32690cc91D4c39d9d3abcBD16989F875707 --rpc-url http://localhost:8545

10100000000000000000


In [17]:
10100000000000000000 / 1e18

10.1

In [12]:
!cast balance 0x70997970C51812dc3A010C7d01b50e0d17dc79C8 --rpc-url http://localhost:8545

10000000000000000000000


In [14]:
!cast send 0x5FC8d32690cc91D4c39d9d3abcBD16989F875707 --value 10000000000000000000 --from 0x70997970C51812dc3A010C7d01b50e0d17dc79C8 --private-key 0x59c6995e998f97a5a0044966f0945389dc9e86dae88c7a8412f4603b6b78690d --rpc-url http://0.0.0.0:8545


blockHash            0x677e647e03c9106984dba45ae2ee0f2ff9f4c4262a245d5e2377e12577071a68
blockNumber          9
contractAddress      
cumulativeGasUsed    21055
effectiveGasPrice    353795046
from                 0x70997970C51812dc3A010C7d01b50e0d17dc79C8
gasUsed              21055
logs                 []
logsBloom            0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
root                 
status               1 (success)
transactionHash      0xa36c07a96ba2c0aaeb8a2f07851ba86838342d58025782def6ba02eba62d6e8b
transactionInd

In [28]:
SOLUTION_ADDRESS = "0x8A791620dd6260079BF849Dc5567aDC3F2FdC318"

ABI = [
    {"type":"function","name":"submitSolution","inputs":[],"outputs":[],"stateMutability":"nonpayable"}
]

solution_contract = w3.eth.contract(address=SOLUTION_ADDRESS, abi=ABI)

# Build the transaction (without gas)
tx = solution_contract.functions.submitSolution().build_transaction({
    'from': OWNER_ADDRESS,
    'nonce': w3.eth.get_transaction_count(OWNER_ADDRESS),
    'gasPrice': w3.to_wei('10', 'gwei')
})

# Estimate gas
try:
    gas_estimate = w3.eth.estimate_gas({
        'from': tx['from'],
        'to': tx['to'],
        'data': tx['data'],
        'value': tx.get('value', 0)
    })
    tx['gas'] = gas_estimate
except Exception as e:
    print(f"Gas estimation failed: {e}")

# Sign and send the transaction
signed_tx = w3.eth.account.sign_transaction(tx, private_key=OWNER_PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
print(f"Solution submitted! Transaction hash: {tx_hash.hex()}")

# Wait for the transaction receipt
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
if receipt.status == 1:
    print("Solution submission was successful!")
else:
    print("Solution submission failed!")

Solution submitted! Transaction hash: 303c6c3282e04b554d1f1bcef4061a2fa4cc78b73afedd5b19b09947b7142336
Solution submission was successful!


In [10]:
!cast balance 0xDc64a140Aa3E981100a9becA4E685f962f0cF6C9 --rpc-url http://localhost:8545

100000000000000000


In [ ]:
100000000000000000 / 1e18

# maybe not sufficient to account to fee or gas price of the send tx, I'm not sure

# TODO INCREASE THE BALANCE OF THE OWNERS AND ALSO THE CONTRACTS

0.1

In [29]:
"""
Now we'll call:

function getProblemPrize(uint256 problemId) external view returns (uint256) {
        return problems[problemId].prize;
    }

    function isProblemSolved(uint256 problemId) external view returns (bool) {
        return problems[problemId].isSolved;
    }

    function problemExists(uint256 problemId) external view returns (bool) {
        return problems[problemId].expectedResultsHash != bytes32(0);
    }

"""

# Call the contract to check if the problem exists
problem_id = 1
problem_exists = contract.functions.problemExists(problem_id).call()
if problem_exists:
    print(f"Problem {problem_id} exists.")
    
    # Get the prize for the problem
    prize = contract.functions.getProblemPrize(problem_id).call()
    print(f"Prize for problem {problem_id}: {Web3.from_wei(prize, 'ether')} ETH")
    
    # Check if the problem is solved
    is_solved = contract.functions.isProblemSolved(problem_id).call()
    print(f"Is problem {problem_id} solved? {'Yes' if is_solved else 'No'}")

Problem 1 exists.
Prize for problem 1: 0 ETH
Is problem 1 solved? Yes
